# Example of Using Savitzky Golay Filter with Solar Images 

This notebook illustrates how to load and use the 3D savitzky golay filters in this data to denoise solar images. 

In [5]:
import savitzkygolay
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from astropy.io import fits
from glob import glob
import os

#### Load Data

First we load the data into one `numpy` array. 

In [11]:
# change the directory to whatever is appropriate
directory = "/media/marcushughes/data/offpoint195/stray_removed/Subtracted_FITS/"
fns = sorted(glob(directory + "*.fits"))
data = []
for fn in fns:
    with fits.open(fn) as f:
        data.append(f[0].data.copy())
data = np.array(data)

#### Smoothing 

Smoothing is relatively easy, assuming you want a uniform order for the polynomial, as opposed to different orders in the different dimensions. You run the command below and wait a few seconds for completion.  

In [ ]:
window_size = 7
poly_order = 3
smoothed = savitzkygolay.filter3D(data, window_size, poly_order)

#### Preview

You can preview the data however you want. The two functions below were designed for the specific dataset. 

In [ ]:
vmin, vmax, power = 0.3, 0.8, 0.25
fig, axs = plt.subplots(ncols=2, sharex=True, sharey=True)
axs[0].imshow(np.power(data[100,:,:], power), vmin=vmin, vmax=vmax, origin='lower')
axs[1].imshow(np.power(smoothed[100,:,:], power), vmin=vmin, vmax=vmax, origin='lower')
fig.show()

In [ ]:
vmin, vmax, power = 0.0, 0.8, 1
fig, axs = plt.subplots(ncols=1, sharex=True, sharey=True)
axs.imshow(np.power(np.abs(data[100,:,:] -smoothed[100,:,:]) , power), vmin=vmin, vmax=vmax, origin='lower')
fig.show()

#### Writing out

Specify the out directory and this will write simple fits images. 

In [14]:
out_dir = "/media/marcushughes/data/offpoint171/filtered3D_7_3/"
for i, fn in enumerate(fns):
    new_fn = out_dir + os.path.basename(fn).replace(".fits", "_filtered.fits")
    fits.writeto(new_fn, smoothed[i,:,:])